beaver,
dolphin, 
otter, 
seal,
whale, 
aquarium fish,
flatfish, 
ray,
shark,
trout, 
orchids,
poppies,
roses,
sunflowers,
tulips, 
bottles,
bowls, 
cans,
cups,
plates, 
apples,
mushrooms,
oranges,
pears,
sweet peppers, 
clock, 
computer keyboard,
lamp,
telephone,
television, 
bed,
chair, 
couch,
table,
wardrobe, 
bee,
beetle,
butterfly,
caterpillar,
cockroach, 
bear,
leopard,
lion,
tiger, 
wolf, 
bridge, 
castle, 
house,
road, 
skyscraper, 
cloud,
forest,
mountain,
plain,
sea, 
camel,
cattle,
chimpanzee, 
elephant,
kangaroo, 
fox,
porcupine,
possum, 
raccoon, 
skunk, 
crab,
lobster, 
snail,
spider,
worm, 
baby,
boy,
girl,
man,
woman, 
crocodile,
dinosaur, 
lizard, 
snake,
turtle, 
hamster,
mouse,
rabbit,
shrew,
squirrel, 
maple,
oak,
palm,
pine,
willow, 
bicycle,
bus,
motorcycle,
pickup truck,
train, 
lawn-mower,
rocket,
streetcar,
tank, 
tractor,
T-shirt/top,
Trouser,
Pullover,
Dress,
Coat,
Sandal,
Shirt,
Sneaker,
Bag,
Ankle boot,

In [26]:
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.datasets import mnist
import numpy as np
import cv2
from tensorflow import keras
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [27]:
#Getting data using built-in tensorflow.keras.datasets API
(input_train_cifar10, target_train_cifar10), (input_test_cifar10, target_test_cifar10) = cifar10.load_data()
(input_train_cifar100, target_train_cifar100), (input_test_cifar100, target_test_cifar100) = cifar100.load_data()
(input_train_mnist, target_train_mnist), (input_test_mnist, target_test_mnist) = mnist.load_data()

In [31]:
#To detect many of the daily life objects, combination of all datasets is required.

train_features_set_1 = [] #Storing images combined from all datasets
train_labels_set_1 = []   #Storing labels combined from all datasets
result = []        
start = 100               #Index for data added from other datasets

#Add CIFAR-100 data to train_features_set_1 after converting to grayscale and removing RGB layer
for i in range(len(input_train_cifar100)):
    train_features_set_1.append(np.mean(input_train_cifar100[i],axis=2).reshape((32,32,1)))
    train_labels_set_1.append(target_train_cifar100[i])
    
#Add CIFAR-10 data to train_features_set_1 after converting to grayscale and removing RGB layer
for i in range(10):
    if i == 1: continue
    result = np.where(target_train_cifar10 == i)
    result = result[0][:500]
    for j in result:
        train_features_set_1.append(np.mean(input_train_cifar10[j],axis=2).reshape((32,32,1)))
        train_labels_set_1.append([start])
    start = start + 1

#Add MNIST-Fashion after resizing to 32 x 32 and converting to (32,32,1) format
for i in range(10):
    result = np.where(target_train_mnist == i)
    result = result[0][:500]
    for j in result:
        train_features_set_1.append(cv2.resize(input_train_mnist[j],dsize=(32,32),interpolation=cv2.INTER_AREA).reshape((32,32,1)))
        train_labels_set_1.append([start])
    start = start + 1

#Convert both arrays to numpy
train_features_set_1 = np.array(train_features_set_1)
train_labels_set_1 = np.array(train_labels_set_1)

#Perform shuffling on arrays. Neural Network gives better accuracy when arrays are shuffled rather than when in order
state = np.random.get_state()
np.random.shuffle(train_features_set_1)
np.random.set_state(state)
np.random.shuffle(train_labels_set_1)

#Observe the classes and samples per class to make sure of the count
print(train_features_set_1.shape)
un,count = np.unique(train_labels_set_1,return_counts=True)
print(un,count)

(59500, 32, 32, 1)
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118] [500 500 500 500 500 500 500 500 500 500 500 500 500 500 500 500 500 500
 500 500 500 500 500 500 500 500 500 500 500 500 500 500 500 500 500 500
 500 500 500 500 500 500 500 500 500 500 500 500 500 500 500 500 500 500
 500 500 500 500 500 500 500 500 500 500 500 500 500 500 500 500 500 500
 500 500 500 500 500 500 500 500 500 500 500 500 500 500 500 500 500 500
 500 500 500 500 500 500 500 500 500 500 500 500 500 500 500 500 500 500
 500 500 500 500 500 500 500 500 500 500 500]


In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=(32,32,1)))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Conv2D(64,(3,3),activation='relu'))
model.add(keras.layers.MaxPooling2D((3,3)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(4*4*64, activation='relu'))
model.add(keras.layers.Dense(119, activation='softmax'))
model.summary()

optimizer = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
ep = 10
model.fit(train_features_set_1, train_labels_set_1, epochs=ep)

Model: "sequential_27"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_54 (Conv2D)           (None, 30, 30, 32)        320       
_________________________________________________________________
max_pooling2d_54 (MaxPooling (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_55 (Conv2D)           (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_55 (MaxPooling (None, 4, 4, 64)          0         
_________________________________________________________________
flatten_27 (Flatten)         (None, 1024)              0         
_________________________________________________________________
dense_66 (Dense)             (None, 1024)              1049600   
_________________________________________________________________
dense_67 (Dense)             (None, 119)             